In [ ]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel
!pip install python-dotenv
# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory
#Install policy engine
!pip install policyengine-uk

In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the token
token = os.environ.get("HUGGING_FACE_TOKEN")

# Now you can use the token
print(f"Token loaded: {token[:4]}...{token[-4:]}")  # Prints first and last 4 chars for verification

Token loaded: hf_S...HLUg


In [13]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd
# LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
# sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
 

In [24]:
# Level 1: 20 GBP pw uplift to child element of UC
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant("2026-01-01"), value=292.81+80.00
    )
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount.update(
        start=instant("2026-01-01"), value=339.00+80.00
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)




In [41]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
year=2026
# Step 1: Calculate in_poverty for everyone
baseline_pov = baseline.calculate("in_relative_poverty_ahc", year, map_to="person", use_weights=True)
ages = baseline.calculate("age", year)
weights = baseline.get_weights("age", year)
# Step 2: Create mask for children (age < 16 or < 18 as appropriate)
is_child = ages < 16  # or < 18 depending on definition
print(ages.min())
# Step 3: Get child-level poverty status and weights
child_poverty = baseline_pov[is_child]
child_weights = weights[is_child]
# Step 4: Calculate total number and percent of children in poverty
num_children = child_weights.sum()
num_children_in_poverty = (child_poverty).sum()
percent_in_poverty = num_children_in_poverty / num_children
print(f"Number of children: {num_children:,.0f}")
print(f"Number of children in poverty: {num_children_in_poverty:,.0f}")
print(f"Percent of children in poverty: {percent_in_poverty:.2%}")
 
# Step 1: Calculate in_poverty for everyone
baseline_pov = reformed.calculate("in_relative_poverty_ahc", year, map_to="person", use_weights=True)
ages = reformed.calculate("age", year)
weights = reformed.get_weights("age", year)
# Step 2: Create mask for children (age < 16 or < 18 as appropriate)
is_child = ages < 16  # or < 18 depending on definition
print(ages.min())
# Step 3: Get child-level poverty status and weights
child_poverty = baseline_pov[is_child]
child_weights = weights[is_child]
# Step 4: Calculate total number and percent of children in poverty
num_children = child_weights.sum()
num_children_in_poverty_after = (child_poverty).sum()
percent_in_poverty_after = num_children_in_poverty / num_children
print(f"Number of children: {num_children:,.0f}")
print(f"Number of children in poverty: {num_children_in_poverty_after:,.0f}")
print(f"Percent of children in poverty: {percent_in_poverty_after:.2%}")
print(f"Reduction in child poverty: {num_children_in_poverty - num_children_in_poverty_after:,.0f}")

0.0
Number of children: 13,668,940
Number of children in poverty: 4,883,989
Percent of children in poverty: 35.73%
0.0
Number of children: 13,668,940
Number of children in poverty: 4,734,371
Percent of children in poverty: 35.73%
Reduction in child poverty: 149,618


In [42]:
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calculate("gov_spending", 2026).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in 2026")
revenue = reformed.calculate("gov_spending", 2029).sum() - baseline.calculate("gov_spending", 2029).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in 2029" )

net_income_baseline = baseline.calculate("household_net_income", 2026)
net_income_reformed = reformed.calculate("household_net_income", 2026)
net_income_baseline_29 = baseline.calculate("household_net_income", 2029)
net_income_reformed_29 = reformed.calculate("household_net_income", 2029)
total_net_income_baseline = baseline.calculate("household_net_income", 2029).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2029).sum()
diff=total_net_income_reformed-total_net_income_baseline
print(f"Total net income in 2029: {diff/1e9:.2f}bn")


This reform would cost £2.97bn in 2026
This reform would cost £1.95bn in 2029
Total net income in 2029: 1.95bn


In [ ]:
diff = net_income_reformed*net_income_reformed.weights - net_income_baseline*net_income_baseline.weights
gainers = (diff > 0).sum()
total=net_income_reformed.sum()-net_income_baseline.sum()
print(f"Avg gain of gainers 2026: {total/gainers}")

2057524.0241095778
Avg gain of gainers 2026: 1442.5434090436418


In [44]:
#Level2:  Work allowance increased to 1000 GBP  pa
def change_tax_parameters(parameters):
    # print(type(parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing))
    # latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing["values"].get("2025-04-01")
    april_2025_date = "2025-04-01"
    latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing(april_2025_date)
    parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing.update(
        start=instant("2026-01-01"), value=latest_value+83.33
    )
    latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing(april_2025_date)
    parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing.update(
        start=instant("2026-01-01"), value=latest_value+83.33
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calc("gov_spending", 2026).sum()
print(f"Cost 2026: £{round(revenue / 1e+9, 1)}bn")
revenue = reformed.calculate("gov_spending", 2029).sum() - baseline.calc("gov_spending", 2029).sum()
print(f"Cost 2029: £{round(revenue / 1e+9, 1)}bn")

Cost 2026: £0.7bn
Cost 2029: £0.4bn


In [ ]:
#Level 2: Change child limit
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.limit.child_count.update(
        start=instant("2026-01-01"), value=inf
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calculate("gov_spending", 2026).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
revenue = reformed.calculate("gov_spending", 2029).sum() - baseline.calculate("gov_spending", 2029).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
#####IMPACT ON CHILD POVERTY
year=2026
def calc_childpov(baseline,year):
    # Step 1: Calculate in_poverty for everyone
    baseline_pov = baseline.calculate("in_relative_poverty_ahc", year, map_to="person", use_weights=True)
    ages = baseline.calculate("age", year)
    weights = baseline.get_weights("age", year)
    # Step 2: Create mask for children (age < 16 or < 18 as appropriate)
    is_child = ages < 16  # or < 18 depending on definition
    print(ages.min())
    # Step 3: Get child-level poverty status and weights
    child_poverty = baseline_pov[is_child]
    child_weights = weights[is_child]
    # Step 4: Calculate total number and percent of children in poverty
    num_children = child_weights.sum()
    num_children_in_poverty = (child_poverty).sum()
    percent_in_poverty = num_children_in_poverty / num_children
    print(f"Number of children: {num_children:,.0f}")
    print(f"Number of children in poverty: {num_children_in_poverty:,.0f}")
    print(f"Percent of children in poverty: {percent_in_poverty:.2%}")
    return num_children_in_poverty
num_children_in_poverty_pre=calc_childpov(baseline,year)
num_children_in_poverty_post=calc_childpov(reformed,year)
print(f"Reduction in child poverty: {num_children_in_poverty_pre - num_children_in_poverty_post:,.0f}")
 

Cost: £1.5bn
Cost: £2.0bn
0.0
Number of children: 15,435,229
Number of children in poverty: 5,344,630
Percent of children in poverty: 34.63%
0.0
Number of children: 15,435,229
Number of children in poverty: 5,180,919
Percent of children in poverty: 33.57%
Reduction in child poverty: 163,711


0.0
Number of children: 13,668,940
Number of children in poverty: 4,883,989
Percent of children in poverty: 35.73%
0.0
Number of children: 13,668,940
Number of children in poverty: 4,723,822
Percent of children in poverty: 35.73%
Reduction in child poverty: 160,167


##RUN 2 CHILD LIMIT REFORM: 

In [ ]:
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "age", "household_net_income", "household_income_decile", "in_poverty", "household_tax", "household_benefits"]
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
difference_person_df = reformed_person_df - baseline_person_df
total_net_income_baseline = baseline.calculate("household_net_income", 2026).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2026).sum()

net_cost = total_net_income_reformed - total_net_income_baseline

print(f"This reform would cost £{net_cost / 1e9:.1f}bn")

In [ ]:

def modify_parameters(parameters):
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_OLD.update(start=instant("2024-01-01"), value=738.82)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_YOUNG.update(start=instant("2024-01-01"), value=618.51)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_OLD.update(start=instant("2024-01-01"), value=528.74)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_YOUNG.update(start=instant("2024-01-01"), value=452.11)
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)

baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2025).sum() - baseline.calc("gov_spending", 2025).sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"


In [ ]:
# Step 1: Make sure to include the weights column in your variables
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "household_weight", "person_weight"]  # Include the weight variable
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, period=2032, use_weights=True).astype(float)
weighted_persons= baseline_person_df["person_weight"].sum()

# Step 2: Create a household-level dataframe by dropping duplicate household IDs
household_df = baseline_person_df.drop_duplicates('household_id')

# Step 3: Calculate the weighted number of households
# Sum the weights column, not the entire dataframe
weighted_households = household_df["household_weight"].sum()

print(f"Total weighted number of households: {weighted_households:,.0f}")
print(f"Total weighted number of people: {weighted_persons:,.0f}")